# Collect Data

In [1]:
import threading
import time
import datetime

import shutil

import pyautogui

from pynput.keyboard import Key, Listener
from pynput import mouse 
import logging
import os




output_dir = f"session_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
print("output_dir", output_dir)


import os

os.makedirs(output_dir)

os.makedirs(f"{output_dir}/images")

# Set up logging to log keystrokes to a file
logging.basicConfig(filename=(f"{output_dir}/keylog.txt"), level=logging.DEBUG, format="%(message)s")


def capture_screenshot(out_dir, key):
    # Capture the screenshot
    screenshot = pyautogui.screenshot()

    output_file = f"{out_dir}/screenshot_{key}.png"
    # Save the screenshot
    screenshot.save(output_file)
    return output_file





global last_ts_now
last_ts_now = datetime.datetime.now()
print("last_ts_now", last_ts_now)

global command_buffer
command_buffer = ""

def heartbeat():
    
    import winsound

    def beep():
        frequency = 500  # Set Frequency To 2500 Hertz
        duration = 777  # Set Duration To 1000 ms == 1 second
        winsound.Beep(frequency, duration)

    sleep_time = 0.5

    while True:
        global last_ts_now
        global command_buffer
        
        
        ts_now = datetime.datetime.now()

        time_difference = ts_now - last_ts_now

        # Convert the difference to milliseconds
        seconds = time_difference.total_seconds()

        

        #term_key = "<key>Key.alt_l  <key>Key.tab"

        #print("seconds", seconds)

        skip_sleep = False
        if seconds > sleep_time:
            if len(command_buffer) > 0:
                ts = ts_now.strftime('%Y-%m-%d_%H-%M-%S.%f')
                
                screenshot_file = f"screen_{ts}"
                
                cb = command_buffer.strip()
                #if term_key not in cb:

                #capture_screenshot(output_dir, screenshot_file)
                file_path = f"{output_dir}/screenshot_current.png"

                if os.path.isfile(file_path):
                    shutil.copyfile(file_path, f"{output_dir}/images/{screenshot_file}.png")
                    logging.info(f"{screenshot_file}\t{str(cb)}")
                    print("Snapshot:", last_ts_now, command_buffer, seconds)
                
                # beep()

                command_buffer = ""
        
            capture_screenshot(output_dir, "current")
        
        if not skip_sleep:
            time.sleep(sleep_time)
        

heartbeat_thread = threading.Thread(target=heartbeat)

heartbeat_thread.start()

# ==========================================================

def update_command_buffer(s):
    if s:
        global command_buffer
        command_buffer += s
        ts_now = datetime.datetime.now()

        global last_ts_now
        last_ts_now = ts_now


def on_move(x, y):
    # print('Pointer moved to {0}'.format(
    #     (x, y)))
    #update_command_buffer(f" <mouse>on_move({x},{y})")
    pass

def on_click(x, y, button, pressed):
    # print('{0} at {1}'.format(
    #     'Pressed' if pressed else 'Released',
    #     (x, y)))
    
    update_command_buffer(f" <mouse>on_click({x},{y},{button},{pressed})")

    # if not pressed:
    #     # Stop listener
    #     return False

def on_scroll(x, y, dx, dy):
    # print('Scrolled {0} at {1}'.format(
    #     'down' if dy < 0 else 'up',
    #     (x, y)))
    update_command_buffer(f" <mouse>on_scroll({x},{y},{dx},{dy})")



def process_on_key(key, pressed):
    s = f"{key}"
    if len(s) > 0 and s[0] == '\'' and s[-1] == '\'':
        s = s[1:-1]

    tag = ""
    if "Key." in s:
        tag = "key"   
        s = f" <{tag}>{s}:{pressed} "
    
    update_command_buffer(s)
    

def on_press(key):

    s = process_on_key(key, True)         
    
    update_command_buffer(s)

def on_release(key):

    s = process_on_key(key, False)
    
    update_command_buffer(s)

def start_key_listener():
    # Start listening to keystrokes
    with Listener(on_press=on_press, on_release=on_release) as listener:
        listener.join()

key_listener_thread = threading.Thread(target=start_key_listener)

key_listener_thread.start()

# Collect events until released
with mouse.Listener(
        on_move=on_move,
        on_click=on_click,
        on_scroll=on_scroll) as listener:
    listener.join()



output_dir session_2024-07-29_20-10-44
last_ts_now 2024-07-29 20:10:44.207440
Snapshot: 2024-07-29 20:10:46.231239  <key>Key.alt_l:True  <key>Key.tab:True  <key>Key.alt_l:False  <key>Key.tab:False  0.961154
Snapshot: 2024-07-29 20:10:47.590530  <key>Key.alt_l:True  <key>Key.tab:True  <key>Key.alt_l:False  <key>Key.tab:False  0.703829
Snapshot: 2024-07-29 20:10:54.644182  <key>Key.alt_l:True  <key>Key.tab:True  <key>Key.alt_l:False  <key>Key.tab:False  0.61422
Snapshot: 2024-07-29 20:10:56.001992  <key>Key.alt_l:True  <key>Key.tab:True  <key>Key.alt_l:False  <key>Key.tab:False  0.685615
Snapshot: 2024-07-29 20:11:05.299618  <mouse>on_click(145,316,Button.left,True) <mouse>on_click(145,316,Button.left,False) 0.731663
Snapshot: 2024-07-29 20:11:07.255612  <mouse>on_click(264,284,Button.left,True) <mouse>on_click(264,284,Button.left,False) 0.740795
Snapshot: 2024-07-29 20:11:10.066079  <mouse>on_click(264,284,Button.left,True) <mouse>on_click(264,284,Button.left,False) <key>Key.shift:True 

### Test the Endpoint

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

def test_slm(image_url = "https://raw.githubusercontent.com/ambianic/fall-detection/main/fall_dataset/fall/74.jpg",
                prompt = "What should be the next git command?"):



    # Request data goes here
    # The example below assumes JSON formatting which may be updated
    # depending on the format your endpoint expects.
    # More information can be found here:
    # https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
    data = {
            "prompt" : f"<|user|>\n<|image_1|>{prompt}<|end|><|assistant|>\n",
            "image_url" : image_url
        }

    body = str.encode(json.dumps(data))

    url = 'https://finetunedphi3endpointfin.northeurope.inference.ml.azure.com/score'
    # Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
    api_key = 'kgFJl7af3C6RadFJRKDNyWVm2HltUbeu'
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")

    # The azureml-model-deployment header will force the request to go to a specific deployment.
    # Remove this header to have the request observe the endpoint traffic rules
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'blue' }

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        print(result)
        return result
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))


test_slm()

In [ ]:

import base64

def encode_image(image_data):
    encoded_string = base64.b64encode(image_data).decode("utf8")
    image_file = 'data:image/jpeg;base64,' + encoded_string
    return image_file

res = ""

with open("C:/Users/antonslutsky/Dev/azureml-quickstart/slm-finetuning/llava-finetune/test/session_2024-07-29_14-41-42/images/screen_2024-07-29_15-39-03.659855.png", "rb") as image_file:
    image_file = encode_image(image_file.read())

    print(image_file)

    res = test_slm(prompt="You are an automation agent that controls keyboard and mouse on a computer screen.  What should be the next keyboard or mouse action?", 
                   image_url=image_file)


In [ ]:



live_screenshot = capture_screenshot(".", "live")

with open(live_screenshot, "rb") as image_file:
    image_file = encode_image(image_file.read())

    print(image_file)

    res = test_slm(prompt="You are an automation agent that controls keyboard and mouse on a computer screen.  What should be the next keyboard or mouse action?", 
                   image_url=image_file)

    res_js = json.loads(res.decode("utf-8"))[0]['0']
    print(res_js)

In [ ]:
res_js